In [1]:
!apt-get update && apt-get upgrade -y
!apt install -y libgl1-mesa-dev
!apt install -y libglib2.0-0

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 336 kB in 1s (244 kB/s)                                   
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libnvinfer-plugin7 libnvinfer7
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree       
Read

In [2]:
!pip install opencv-python
!pip install tqdm
!pip install scikit-learn

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!nvidia-smi

Sun Dec 11 19:55:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   33C    P8    17W / 340W |   8483MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from IPython.display import Image, display

In [5]:
from matplotlib import pyplot as plt

In [6]:
import cv2
from PIL import ImageFile
from PIL import Image

In [7]:
import keras
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

In [8]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [9]:
import random
import tqdm
import glob

In [10]:
from datetime import datetime

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
import sys
import pathlib
import os
import shutil
import math

In [13]:
import numpy as np

In [14]:
#14 for GeForce RTX 3080 10GB
#55 for V100 40GB
images_per_batch = 14
window_width = 256
window_height = 256
model_name="scannoise_remover0"
make_noised_images = "no"
make_image_split = "no"
noise_algorithm = "new"
epochs = 100
images_per_bulk = images_per_batch * 100
validation_percent = 0.1
restart_epoch = 0
testimages_max = 3000
if restart_epoch != 0:
    restart = "yes"
else:
    restart = "no"

In [15]:
def add_linenoise(filename, out_dirname):
    sigma = 0.35
    max_thickness = 26
    min_thickness = 10

    image = cv2.imread(filename)
    height, width, c = image.shape
    output_filename_woext = os.path.splitext(os.path.basename(filename))[0]
    output_dir = os.path.split(os.path.dirname(filename))
    os.makedirs(out_dirname +"/" + output_dir[-1], exist_ok=True)

    randline_img = image.copy()
    randline_img = np.array(randline_img, dtype='float32')
    orig_img = randline_img.copy()

    thickness=random.randint(1,max_thickness/2)*2+1
    rand_x=random.randint(      thickness - int(thickness/2) - 1, width - int(thickness/2) - 1)

    #print("noise center", rand_x)
    #print("noise thickness", thickness)

    __col = random.randint(0,7)
    _r = __col % 2
    _g = (__col >> 1) % 2
    _b = (__col >> 2) % 2
    _flag = random.randrange(-1,2,2)
    center_high = 18
    center_low = 6
    variance_high = 3
    variance_low = 1
    weight_center = random.randrange(center_low,center_high)/100.
    weight_width = random.randrange(variance_low,variance_high)/100.

    while True:
        for __x in range(thickness):
            _x = __x - int(thickness/2)
            distance_from_center = - (int(thickness/2) - abs(_x) ) / int(thickness/2) + 1
            gauss_dist = math.exp ( - (distance_from_center) **2  / (2.0 * sigma **2 ) )
            for _y in range(height):
                _col = randline_img[_y, rand_x + _x]
                weight = random.triangular(weight_center-weight_width, weight_center+weight_width
                                                                        , weight_center)
                _col[0] = (_r * 255.) * gauss_dist * _flag * weight + _col [0]
                _col[1] = (_g * 255.) * gauss_dist * _flag * weight + _col [1]
                _col[2] = (_b * 255.) * gauss_dist * _flag * weight + _col [2]
                if _col[0] > 255.:
                    _col[0] = 255.
                if _col[1] > 255.:
                    _col[1] = 255.
                if _col[2] > 255.:
                    _col[2] = 255.
                if _col[0] < 0.:
                    _col[0] = 0.
                if _col[1] < 0.:
                    _col[1] = 0.
                if _col[2] < 0.:
                    _col[2] = 0.
                randline_img[_y, rand_x + _x] = _col
        randline_img = np.array(randline_img, dtype='int16')
        orig_img = np.array(orig_img, dtype='int16')
        randline_img_rgb = randline_img[:,:,0]+randline_img[:,:,1]+randline_img[:,:,2]
        orig_img_rgb = orig_img[:,:,0]+orig_img[:,:,1]+orig_img[:,:,2]

        diff = np.abs(randline_img_rgb - orig_img_rgb)
        diff_max = diff.max()
        diff_sum = diff.sum()
        if diff_sum < 1200 or diff_max < 10:
            #print(output_filename_woext, np.abs(diff).max())
            #print("no diff")
            __col = random.randint(0,7)
            _r = __col % 2
            _g = (__col >> 1) % 2
            _b = (__col >> 2) % 2
            _flag = random.randrange(-1,2,2)
            weight_center = random.randrange(center_low,center_high)/100.
            weight_width = random.randrange(variance_low,variance_high)/100.
            randline_img = image.copy()
            randline_img = np.array(randline_img, dtype='float32')
            orig_img = randline_img.copy()
            continue
        else:
            #print("absmax, sum, _flag, weight_center, weight_width, thickness",  output_filename_woext, diff_max, diff_sum, _flag, weight_center, weight_width, thickness)
            #print("ok")
            randline_img = np.array(randline_img, dtype='uint8')
#            if _flag > 1:
#                diff = np.array(orig_img - randline_img, dtype='uint8')
#            else:
#                diff = np.array(randline_img - orig_img, dtype='uint8')
            cv2.imwrite(out_dirname +"/" + output_dir[-1] + "/" + output_filename_woext+".png",randline_img)
#            cv2.imwrite(out_dirname +"/" + output_dir[-1] + "/" + output_filename_woext+".diff.png",diff)
            break
    return (filename, rand_x)

In [16]:
def add_linenoise_old(filename, out_dirname):
    sigma = 0.35
    max_thickness = 12
    min_thickness = 3
    image = cv2.imread(filename)
    height, width, c = image.shape
    
    thickness=random.randint(1,max_thickness/2)*2+1
    rand_x=random.randint(      thickness - int(thickness/2) - 1, width - int(thickness/2) - 1)
    randline_img = image.copy()
    randline_img = np.array(randline_img, dtype='float32')
    
    output_filename_woext = os.path.splitext(os.path.basename(filename))[0]
    output_dir = os.path.split(os.path.dirname(filename))
    os.makedirs(out_dirname +"/" + output_dir[-1], exist_ok=True) 
    
    #print("noise center", rand_x)
    #print("noise thickness", thickness)

    __col = random.randint(0,7)
    _r = __col % 2
    _g = (__col >> 1) % 2
    _b = (__col >> 2) % 2

    for __x in range(thickness):
        _x = __x - int(thickness/2)
        distance_from_center = - (int(thickness/2) - abs(_x) ) / int(thickness/2) + 1
        gauss_dist = math.exp ( - (distance_from_center) **2  / (2.0 * sigma **2 ) )
        for _y in range(height):
            _col = randline_img[_y, rand_x + _x]
            weight = random.triangular(0.1, 1.0, 0.3)
            _col[0] = (_r * 255) * gauss_dist * weight + _col [0]
            _col[1] = (_g * 255) * gauss_dist * weight + _col [1]
            _col[2] = (_b * 255) * gauss_dist * weight + _col [2]
            if _col[0] > 255:
                _col[0] = 255
            if _col[1] > 255:
                _col[1] = 255
            if _col[2] > 255:
                _col[2] = 255
            if _col[0] < 0:
                _col[0] = 0
            if _col[1] < 0:
                _col[1] = 0
            if _col[2] < 0:
                _col[2] = 0
            randline_img[_y, rand_x + _x] = _col
    cv2.imwrite(out_dirname +"/" + output_dir[-1] + "/" + output_filename_woext+".png",randline_img)
    return (filename, rand_x)

In [17]:
def ImgSplit(filename, out_dirname):
    height = window_height
    width = window_width
    
    im = cv2.imread(filename)
    output_filename_woext = os.path.splitext(os.path.basename(filename))[0]
    output_dir = os.path.split(os.path.dirname(filename))
    os.makedirs(out_dirname +"/" + output_dir[-1], exist_ok=True) 
    img_height, img_width = im.shape[:2]
    # do not make a patch when image patch is too small
    if (img_height < window_height) or (img_width < window_width):
        return
    split_height = int(img_height / height) 
    split_width = int(img_width / width)
    for h1 in range(split_height+1):
        for w1 in range(split_width+1):
            w2 = w1 * width
            h2 = h1 * height
            _h_start = h2
            _h_end   = height+h2-1
            _w_start = w2
            _w_end   = width+w2-1
            if h1 == split_height:
                _h_start = img_height-height
                _h_end   = img_height-1
            if w1 == split_width:
                _w_start = img_width-width
                _w_end   = img_width-1
            c = im[_h_start:_h_end+1, _w_start:_w_end+1]
            if (_h_end+1 - _h_start) == height and (_w_end+1 - _w_start) == width:
                cv2.imwrite(out_dirname +"/" + output_dir[-1] + "/" + output_filename_woext + "_" + str(h1).zfill(3) + "_" + str(w1).zfill(3) +".png",c)


In [18]:
if make_image_split != "no":
    print("making testimages")
    filenames_list=glob.glob('bookscan_linefix/groundtruth/[A,B,C,D,E,F]/*.jpg')
    try:
        shutil.rmtree('clean')
    except OSError as e:
        pass    
    pathlib.Path('clean').mkdir()
    
    for _filename in tqdm.tqdm(filenames_list):
        ImgSplit(_filename, out_dirname = "clean")
else:
    print("do not make testimages")

do not make testimages


In [19]:
if make_noised_images != "no":
    print("making noised images")
    try:
        shutil.rmtree('noised')       
    except OSError as e:
        pass
    pathlib.Path('noised').mkdir()
    filenames_list=glob.glob('clean/*/*.png')
    for _filename in tqdm.tqdm(filenames_list):
        if noise_algorithm == "old":
            #print("using old noise algorithm")
            pp = add_linenoise_old(_filename, out_dirname = "noised")
        else:
            #print("using new noise algorithm")
            pp = add_linenoise(_filename, out_dirname = "noised")

In [20]:
filenames_train_list=sorted(glob.glob('noised/*/*.png'))
filenames_target_list=sorted(glob.glob('clean/*/*.png'))
counter_train = len(filenames_train_list)
counter_target = len(filenames_target_list)
if (counter_train != counter_target):
    print("something wrong in num of pics of target and trains")
    exit(1)
numof_train = int(counter_target * (1.0-validation_percent))
numof_validation = counter_target - numof_train
print("total images",counter_target)
print("training images: ", numof_train)
print("validation images: ", numof_validation)

total images 673596
training images:  606236
validation images:  67360


In [21]:
def get_batch(_X_train, _Y_train, _images_per_batch):
    print("getting a batch")
    print("images per batch", _images_per_batch)
    numof_batchs = len(_X_train)//_images_per_batch
    print("Number of batches", numof_batchs)

    i = 0
    while (i < numof_batchs):
        print("doing a batch of ", i, "/", numof_batchs)
        X_batch = []
        Y_batch = []
        Y_batch = _Y_train[(i * _images_per_batch):(i * _images_per_batch + _images_per_batch)]
        X_batch = _X_train[(i * _images_per_batch):(i * _images_per_batch + _images_per_batch)]
        i += 1
        yield X_batch, Y_batch

In [22]:
def get_bulk(_X_train, _Y_train, _images_per_bulk):
    print("images per bluk", _images_per_bulk)
    numof_bulks = len(_X_train)// _images_per_bulk
    print("Number of bulks", numof_bulks)
    i = 0
    while (i < numof_bulks):
        print("doing a bulk of ", i, "/", numof_bulks)
        X_train_bulk = []
        Y_train_bulk = []
        X_train_bulk_name = []
        Y_train_bulk_name = []
        print("getting a bluk...start ")
        X_train_name = _X_train[(i * _images_per_bulk):(i * _images_per_bulk + _images_per_bulk)]
        Y_train_name = _Y_train[(i * _images_per_bulk):(i * _images_per_bulk + _images_per_bulk)]
        X_train_bulk = np.array([np.array(Image.open(file)) for file in X_train_name])
        Y_train_bulk = np.array([np.array(Image.open(file)) for file in Y_train_name])
        print("getting a bluk...done ")

        i += 1
        yield X_train_bulk, Y_train_bulk

In [23]:
# split noised data to training and test (validation) sets
X_train, x_test= train_test_split(filenames_train_list, test_size=validation_percent, random_state=42) #from sklearn
Y_train = [s.replace('noised', 'clean') for s in X_train]
y_test = [s.replace('noised', 'clean') for s in x_test]

In [24]:
input_img = Input(shape=(window_height, window_width, 3))
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = Activation('relu')(x)

for i in range(15):
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

x = Conv2D(3, (3, 3), padding='same')(x)
output_img = Activation('tanh')(x)

In [25]:
model = Model(input_img, output_img)
model.compile(optimizer='adam', loss='mean_squared_error')

In [26]:
import gc

if restart == "yes":
    print("restarting from epoch #", restart_epoch)
    model.load_weights(model_name + '.' + str(restart_epoch-1).zfill(3) +'.h5')
    
for epoch in range(restart_epoch, epochs):
    print("=" * 80)
    print(epoch, "/", epochs, " EPOCHS")
    acc = []

    for X_train_bulk, Y_train_bulk in get_bulk(X_train, Y_train, images_per_bulk):
        for X_batch, Y_batch in get_batch(X_train_bulk, Y_train_bulk, images_per_batch):
            print('Training start for a batch')
            # normalize data
            X_batch = X_batch.astype('float32')
            X_batch /= 255
            Y_batch = Y_batch.astype('float32')
            Y_batch /= 255
            model.train_on_batch(X_batch, Y_batch)
            score = model.evaluate(X_batch, Y_batch)
            print("batch accuracy:", score)
            acc.append(score)#
        tf.keras.backend.clear_session()
        gc.collect()
    print("Train accuracy (mean)", np.mean(acc))
    print("Train accuracy (max)", np.max(acc))
    model.save(model_name + '.' + str(epoch).zfill(3) +'.h5')

    #with tf.device('/cpu:0'):
    #    loss, acc = model.evaluate(X_test, Y_test, verbose=2)
    #    print("Loss and accuracy", loss, acc)

0 / 100  EPOCHS
images per bluk 1400
Number of bulks 433
doing a bulk of  0 / 433
getting a bluk...start 
getting a bluk...done 
getting a batch
images per batch 14
Number of batches 100
doing a batch of  0 / 100
Training start for a batch


UnimplementedError: Graph execution error:

Detected at node 'model/activation/Relu' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 505, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyncio/base_events.py

In [ ]:
X_test = np.array([np.array(Image.open(file)) for file in x_test[0:testimages_max]])
Y_test = np.array([np.array(Image.open(file)) for file in y_test[0:testimages_max]])
X_test = X_test.astype('float32')
X_test /= 255
Y_test = Y_test.astype('float32')
Y_test /= 255

In [ ]:
# definition to show original image and reconstructed image
def showOrigDec(orig, noise, denoise, num=5):
    import matplotlib.pyplot as plt
    n = num
    plt.figure()  #figsize=(20, 6))

    for i in range(n):
        # display original
        ax = plt.subplot(5, n, i+1)
        _orig = np.copy(orig[i])
        _orig *= 255
        _orig = _orig.astype('uint8')
        plt.imshow(_orig.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display noisy image
        ax = plt.subplot(5, n, i +1 + n)
        _noise = np.copy(noise[i])
        _noise *= 255
        _noise = _noise.astype('uint8')
        plt.imshow(_noise.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display diff between the original and the noisy image
        ax = plt.subplot(5, n, i +1 + n + n)
        im_diff = _noise - _orig
        im_diff_center = np.floor_divide(im_diff, 2) + 128      
        im_diff_center = im_diff_center.astype('uint8')        
        plt.imshow(im_diff_center.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display the denoised image
        ax = plt.subplot(5, n, i +1 + n + n + n)
        _denoise = np.copy(denoise[i])
        _denoise *= 255
        _denoise = _denoise.astype('uint8')
        plt.imshow(_denoise.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display diff between the original and the denoised image
        ax = plt.subplot(5, n, i +1 + n + n + n + n)
        _denoise = np.copy(denoise[i])
        _denoise *= 255
        _orig = np.copy(orig[i])
        _orig *= 255
        im_diff = _denoise - _orig
        im_diff_center = np.floor_divide(im_diff, 2) + 128      
        im_diff_center = im_diff_center.astype('uint8')   
        plt.imshow(im_diff_center.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()

In [ ]:
model = load_model('scannoise_remover.' + str(testimage_epoch).zfill(3) + '.h5')
model.summary()
print("testimages: ", testimages_max)
with tf.device('/gpu:1'):
    magazine_test = model.predict(X_test[0:testimages_max])

In [ ]:
showOrigDec(X_test, Y_test, magazine_test)